In [1]:
!curl -L -o ./parking-space-detection-dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/trainingdatapro/parking-space-detection-dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 42.3M  100 42.3M    0     0  46.9M      0 --:--:-- --:--:-- --:--:-- 58.3M


In [2]:
!unzip parking-space-detection-dataset.zip -d parking-space-detection-dataset

Archive:  parking-space-detection-dataset.zip
  inflating: parking-space-detection-dataset/annotations.xml  
  inflating: parking-space-detection-dataset/boxes/0.png  
  inflating: parking-space-detection-dataset/boxes/1.png  
  inflating: parking-space-detection-dataset/boxes/10.png  
  inflating: parking-space-detection-dataset/boxes/11.png  
  inflating: parking-space-detection-dataset/boxes/12.png  
  inflating: parking-space-detection-dataset/boxes/13.png  
  inflating: parking-space-detection-dataset/boxes/14.png  
  inflating: parking-space-detection-dataset/boxes/15.png  
  inflating: parking-space-detection-dataset/boxes/17.png  
  inflating: parking-space-detection-dataset/boxes/18.png  
  inflating: parking-space-detection-dataset/boxes/19.png  
  inflating: parking-space-detection-dataset/boxes/2.png  
  inflating: parking-space-detection-dataset/boxes/20.png  
  inflating: parking-space-detection-dataset/boxes/21.png  
  inflating: parking-space-detection-dataset/boxes/22.

# Parsing

In [1]:
import xml.etree.ElementTree as ET
from sklearn.model_selection import KFold, train_test_split
from pathlib import Path
import yaml
import shutil

In [2]:
dataset_path = Path('parking-space-detection-dataset')
root = ET.parse(dataset_path / 'annotations.xml').getroot()

In [3]:
imgs = []
for image_tag in root.findall('image'):
    image_name = image_tag.get('name')
    image_width = int(image_tag.get('width'))
    image_height = int(image_tag.get('height'))

    polygons = []
    for polygon_tag in image_tag.findall('polygon'):
        polygon_label = polygon_tag.get('label')
        polygon_points = polygon_tag.get('points')
        polygons.append((polygon_label, polygon_points))

    imgs.append((image_name, polygons, (image_width, image_height)))

In [4]:
classes = set()

for img in imgs:
    for polygon in img[1]:
        classes.add(polygon[0])

classes = list(classes)
class_to_idx = {classes[i]: i for i in range(len(classes))}
idx_to_class = {v: k for k, v in class_to_idx.items()}

# replace the label with the index
imgs = [(img[0], [(class_to_idx[polygon[0]], polygon[1]) for polygon in img[1]], img[2]) for img in imgs]

In [5]:
ksplit = 4
kf = KFold(n_splits=ksplit, shuffle=True, random_state=42)
kfolds = list(kf.split(imgs))

folds = [f"split_{n}" for n in range(1, ksplit + 1)]

save_path = Path(dataset_path / f"{ksplit}-Fold_Cross-val")
save_path.mkdir(parents=True, exist_ok=True)

ds_yamls = []

for n, (train_idx, val_idx) in enumerate(kfolds):
    print("RUNNING SPLIT ", n + 1)
    train_data = [imgs[i] for i in train_idx]
    val_data = [imgs[i] for i in val_idx]

    split_dir = save_path / folds[n]
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "labels").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "labels").mkdir(parents=True, exist_ok=True)

    dataset_yaml = split_dir / f"dataset.yaml"
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, "w") as ds_y:

        yaml.safe_dump(
            {
                "path": split_dir.resolve().as_posix(),
                "train": "./train",
                "val": "./val",
                "names": idx_to_class,
            },
            ds_y,
        )

    #for image_data, split in zip([train_data, val_data], ["train", "val"]):
    for image_data, split in zip([train_data, val_data], ["train", "val"]):
        for img in image_data:
            shutil.copy(
                dataset_path / img[0],
                split_dir / split / img[0],
            )

            txt_path = split_dir / split / img[0].replace(".png", ".txt").replace('image', 'label')
            with open(txt_path, "w") as f:
                for polygon in img[1]:
                    polygon_parsed = polygon[1].split(";")
                    for i, point in enumerate(polygon_parsed):
                        x, y = point.split(",")
                        x = float(x) / img[2][0]
                        y = float(y) / img[2][1]
                        polygon_parsed[i] = f"{x} {y}"
                    polygon_parsed = " ".join(polygon_parsed)
                    f.write(f"{polygon[0]} {polygon_parsed}\n")

                

RUNNING SPLIT  1
RUNNING SPLIT  2
RUNNING SPLIT  3
RUNNING SPLIT  4


In [8]:
train_data, val_data = train_test_split(
    imgs,
    train_size=0.8,
    random_state=42,
    shuffle=True
)

save_path = Path(dataset_path / "80-20_Split")
save_path.mkdir(parents=True, exist_ok=True)
(save_path / "train" / "images").mkdir(parents=True, exist_ok=True)
(save_path / "train" / "labels").mkdir(parents=True, exist_ok=True)
(save_path / "val" / "images").mkdir(parents=True, exist_ok=True)
(save_path / "val" / "labels").mkdir(parents=True, exist_ok=True)

dataset_yaml = save_path / f"dataset.yaml"

with open(dataset_yaml, "w") as ds_y:

    yaml.safe_dump(
        {
            "path": save_path.resolve().as_posix(),
            "train": "./train",
            "val": "./val",
            "names": idx_to_class,
        },
        ds_y,
    )

for image_data, split in zip([train_data, val_data], ["train", "val"]):
    for img in image_data:
        shutil.copy(
            dataset_path / img[0],
            save_path / split / img[0],
        )

        txt_path = save_path / split / img[0].replace(".png", ".txt").replace('image', 'label')
        with open(txt_path, "w") as f:
            for polygon in img[1]:
                polygon_parsed = polygon[1].split(";")
                for i, point in enumerate(polygon_parsed):
                    x, y = point.split(",")
                    x = float(x) / img[2][0]
                    y = float(y) / img[2][1]
                    polygon_parsed[i] = f"{x} {y}"
                polygon_parsed = " ".join(polygon_parsed)
                f.write(f"{polygon[0]} {polygon_parsed}\n")